In [1]:
# development_calibration.ipynb
import cv2
import numpy as np
import glob
import os
import json
import matplotlib.pyplot as plt
import utility.stereo_utils as su
from IPython.display import display, Image


In [2]:
# ====== CALIBRATION SETTINGS ======
CHESSBOARD_SIZE = (8, 5)  # (columns, rows) - INTERNAL corners
SQUARE_SIZE = 0.025       # 2.5cm squares
CALIB_IMAGES_DIR = "calibration_images/"
OUTPUT_JSON = "development_calibration.json"

# ====== PATHS ======
os.makedirs(CALIB_IMAGES_DIR, exist_ok=True)
os.makedirs("development/steps", exist_ok=True)

# ====== OBJECT POINTS ======
objp = np.zeros((CHESSBOARD_SIZE[0] * CHESSBOARD_SIZE[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESSBOARD_SIZE[0], 0:CHESSBOARD_SIZE[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE

====== MAIN CALIBRATION ======

In [3]:
def calibrate_monocular(image_paths, side_name):
    objpoints, imgpoints = [], []
    img_shape = None
    
    for idx, fname in enumerate(image_paths):
        img = cv2.imread(fname)
        if img is None:
            print(f"⚠️ Skipping invalid image: {fname}")
            continue
            
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Try different detection methods
        ret, corners = cv2.findChessboardCornersSB(
            gray, CHESSBOARD_SIZE, 
            flags=cv2.CALIB_CB_NORMALIZE_IMAGE|cv2.CALIB_CB_EXHAUSTIVE
        )
        
        if not ret:  # Fallback to standard method
            ret, corners = cv2.findChessboardCorners(
                gray, CHESSBOARD_SIZE,
                flags=cv2.CALIB_CB_ADAPTIVE_THRESH
            )
        
        if ret:
            corners = cv2.cornerSubPix(
                gray, corners, (11,11), (-1,-1),
                criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
            )
            
            if img_shape is None:
                img_shape = gray.shape[::-1]
                
            objpoints.append(objp)
            imgpoints.append(corners)
            
            # Visual verification
            vis = cv2.drawChessboardCorners(img.copy(), CHESSBOARD_SIZE, corners, ret)
            cv2.imwrite(f"development/steps/{side_name.lower()}_corners_{idx:02d}.png", vis)
        else:
            print(f"❌ Chessboard not found in {os.path.basename(fname)}")
    
    if not objpoints:
        raise ValueError(f"No chessboards detected in {side_name} images!")
    
    # Simplified calibration
    ret, mtx, dist, _, _ = cv2.calibrateCamera(
        objpoints, imgpoints, img_shape, None, None
    )
    
    print(f"{side_name} Calibration Error: {ret:.3f} pixels")
    return mtx, dist, objpoints, imgpoints

In [4]:
# Load images
left_images = sorted(glob.glob("development/left/*.jpg"))
right_images = sorted(glob.glob("development/right/*.jpg"))
stereo_images = sorted(glob.glob("development/together/*.jpg"))

In [5]:
# Add this before calibration
print(f"Found {len(left_images)} left images and {len(right_images)} right images")
for img_path in left_images[:3]:  # Check first 3 images
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Failed to load: {img_path}")
    else:
        print(f"✓ {img_path} - {img.shape}")

Found 28 left images and 35 right images
✓ development/left\left_00.jpg - (480, 640, 3)
✓ development/left\left_01.jpg - (480, 640, 3)
✓ development/left\left_02.jpg - (480, 640, 3)


In [6]:
# Monocular Calibration
print("=== MONOCULAR CALIBRATION ===")
mtxL, distL, objL, imgL = calibrate_monocular(left_images, "LEFT")
mtxR, distR, objR, imgR = calibrate_monocular(right_images, "RIGHT")

=== MONOCULAR CALIBRATION ===
LEFT Calibration Error: 0.234 pixels
RIGHT Calibration Error: 0.324 pixels


In [8]:
# ===== PROPER STEREO CALIBRATION =====
print("\n=== RUNNING STEREO CALIBRATION ===")

# Load synchronized stereo pairs
stereo_images = sorted(glob.glob("development/together/*.jpg"))
objpoints_stereo = []
imgpoints_left = []
imgpoints_right = []

for idx, fname in enumerate(stereo_images):
    img = cv2.imread(fname)
    h, w = img.shape[:2]
    left = img[:, :w//2]
    right = img[:, w//2:]
    
    grayL = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(right, cv2.COLOR_BGR2GRAY)
    
    # Detect corners in BOTH images
    retL, cornersL = cv2.findChessboardCornersSB(grayL, (8,5), None)
    retR, cornersR = cv2.findChessboardCornersSB(grayR, (8,5), None)
    
    if retL and retR:
        # Refine corners
        cornersL = cv2.cornerSubPix(grayL, cornersL, (11,11), (-1,-1),
                                   criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        cornersR = cv2.cornerSubPix(grayR, cornersR, (11,11), (-1,-1),
                                    criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        
        objpoints_stereo.append(objp)
        imgpoints_left.append(cornersL)
        imgpoints_right.append(cornersR)
        
        # Visualization
        vis = np.hstack([
            cv2.drawChessboardCorners(left.copy(), (8,5), cornersL, retL),
            cv2.drawChessboardCorners(right.copy(), (8,5), cornersR, retR)
        ])
        cv2.imwrite(f"development/steps/stereo_pair_{idx:02d}.png", vis)

print(f"Found {len(objpoints_stereo)} valid stereo pairs")


flags = cv2.CALIB_FIX_INTRINSIC  # Preserves mtxL, mtxR, distL, distR

ret, mtxL, distL, mtxR, distR, R, T, E, F = cv2.stereoCalibrate(
    objpoints_stereo,
    imgpoints_left,    # Left camera points
    imgpoints_right,   # Right camera points
    mtxL, distL,       # From left monocular calibration
    mtxR, distR,       # From right monocular calibration
    grayL.shape[::-1], # Image dimensions
    criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5),
    flags=flags        # Critical fix!
)

print("\n=== RESULTS ===")
print(f"Calibration Error: {ret:.3f} pixels (should be < 0.5)")
print(f"Translation (Baseline): X={T[0][0]:.4f}m, Y={T[1][0]:.4f}m, Z={T[2][0]:.4f}m")
print(f"Baseline Norm: {np.linalg.norm(T):.4f}m (expected: 0.120m)")

# Verify T direction makes sense
print("\nTranslation Vector Analysis:")
print(f"X (left-right): {T[0][0]:.4f}m (should be ~0.12)")
print(f"Y (vertical): {T[1][0]:.4f}m (should be ~0)")
print(f"Z (depth): {T[2][0]:.4f}m (should be ~0)")


=== RUNNING STEREO CALIBRATION ===
Found 24 valid stereo pairs

=== RESULTS ===
Calibration Error: 0.353 pixels (should be < 0.5)
Translation (Baseline): X=-0.1117m, Y=0.0059m, Z=-0.0693m
Baseline Norm: 0.1316m (expected: 0.120m)

Translation Vector Analysis:
X (left-right): -0.1117m (should be ~0.12)
Y (vertical): 0.0059m (should be ~0)
Z (depth): -0.0693m (should be ~0)


In [9]:
# Rectification
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
    mtxL, distL, mtxR, distR,
    grayL.shape[::-1], R, T,
    alpha=0.9,  # Preserve 90% of image
    flags=cv2.CALIB_ZERO_DISPARITY
)

In [10]:
# Save results
calib_data = {
    "mtx_left": mtxL.tolist(),
    "dist_left": distL.tolist(),
    "mtx_right": mtxR.tolist(),
    "dist_right": distR.tolist(),
    "R": R.tolist(),
    "T": T.tolist(),
    "E": E.tolist(),
    "F": F.tolist(),
    "R1": R1.tolist(),
    "R2": R2.tolist(),
    "P1": P1.tolist(),
    "P2": P2.tolist(),
    "Q": Q.tolist()
}

with open(OUTPUT_JSON, 'w') as f:
    json.dump(calib_data, f, indent=4)

print(f"\nCalibration saved to {OUTPUT_JSON}")


Calibration saved to development_calibration.json


In [11]:
# Load calibration
with open("development_calibration.json") as f:
    calib = json.load(f)

# Verify parameters
print("=== CALIBRATION VALIDATION ===")
print(f"Left Camera Focal Length: {calib['mtx_left'][0][0]:.1f}, {calib['mtx_left'][1][1]:.1f}")
print(f"Right Camera Focal Length: {calib['mtx_right'][0][0]:.1f}, {calib['mtx_right'][1][1]:.1f}")
print(f"Baseline Distance: {np.linalg.norm(calib['T']):.3f} meters")

# Test undistortion
test_img = cv2.imread("development/together/together_75.jpg")
left, right = su.Split_Stereo_Frame(test_img)

left_undist = cv2.undistort(left, np.array(calib['mtx_left']), np.array(calib['dist_left']))
right_undist = cv2.undistort(right, np.array(calib['mtx_right']), np.array(calib['dist_right']))

cv2.imwrite("validation_left.jpg", np.hstack((left, left_undist)))
cv2.imwrite("validation_right.jpg", np.hstack((right, right_undist)))



=== CALIBRATION VALIDATION ===
Left Camera Focal Length: 655.7, 654.2
Right Camera Focal Length: 532.2, 531.3
Baseline Distance: 0.132 meters


True

In [12]:
# Current reported baseline
T = np.array(calib['T'])
calculated_baseline = np.linalg.norm(T)
print(f"Calibrated Baseline: {calculated_baseline:.3f}m (Expected: 0.120m)")
print(f"Discrepancy: {abs(calculated_baseline-0.12)*100:.1f}cm")

# Check translation vector direction
print("\nTranslation Vector Components:")
print(f"X: {T[0][0]:.4f}m (should match left/right direction)")
print(f"Y: {T[1][0]:.4f}m (typically small)")
print(f"Z: {T[2][0]:.4f}m (typically near zero)")

Calibrated Baseline: 0.132m (Expected: 0.120m)
Discrepancy: 1.2cm

Translation Vector Components:
X: -0.1117m (should match left/right direction)
Y: 0.0059m (typically small)
Z: -0.0693m (typically near zero)
